## KNN 的補值方法

In [1]:
# library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display
import sklearn

%matplotlib inline



## 調整不同的K 和 weights 的效果

### K=1,weights= uniform

In [2]:
ori_data={'sex': ['Male','Male','Male','Male','Male','Female','Female','Female','Female','Female'],
     'age':[23,40,5,30,1,40,16,27,43,8],
     'height':[180,170,100,176,70,160,170,166,155,np.NAN],
     'weight':[100,68,20,np.NAN,10,np.NAN,50,58,58,17]}
 
ori_data=pd.DataFrame(ori_data)
display(ori_data)

,sex,age,height,weight
0,Male,23,180.0,100.0
1,Male,40,170.0,68.0
2,Male,5,100.0,20.0
3,Male,30,176.0,NaN
4,Male,1,70.0,10.0
5,Female,40,160.0,NaN
6,Female,16,170.0,50.0
7,Female,27,166.0,58.0
8,Female,43,155.0,58.0
9,Female,8,NaN,17.0


### Step1:  類別轉數值型資料

In [3]:
data=ori_data.copy() #複製一份資料給 data
#ValueError: could not convert string to float: 'Male', sklearn 中的 KNN 只能處理數值型態
#轉型 
#透過 preprocessing 轉換
from sklearn import preprocessing
#下面兩行程式，讓 Male=1, FeMale-0.
le = preprocessing.LabelEncoder()
data['sex']=le.fit_transform(data['sex'])
display(data)



,sex,age,height,weight
0,1,23,180.0,100.0
1,1,40,170.0,68.0
2,1,5,100.0,20.0
3,1,30,176.0,NaN
4,1,1,70.0,10.0
5,0,40,160.0,NaN
6,0,16,170.0,50.0
7,0,27,166.0,58.0
8,0,43,155.0,58.0
9,0,8,NaN,17.0


### Step 2 :計算資料點間的距離

In [4]:
from sklearn.metrics.pairwise import nan_euclidean_distances
display(pd.DataFrame(nan_euclidean_distances(data)))
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.nan_euclidean_distances.html?highlight=nan_euclidean

#所以從這各表中，我們可以發現 第九個位置的點，和 2 最接近。

,0,1,2,3,4,5,6,7,8,9
0,0.000000,37.589892,114.560028,9.309493,143.819331,30.331502,51.478151,44.463468,52.820451,97.399521
1,37.589892,0.000000,91.809586,13.466007,122.004098,11.604597,30.016662,16.911535,18.303005,69.531767
2,114.560028,91.809586,0.000000,92.383260,31.874755,80.216374,76.954532,79.277992,76.902536,5.033223
3,9.309493,13.466007,92.383260,0.000000,126.896283,21.817424,17.625739,12.110601,28.542366,31.144823
4,143.819331,122.004098,31.874755,126.896283,0.000000,113.266647,108.747414,110.440029,106.273233,11.489125
5,30.331502,11.604597,80.216374,21.817424,113.266647,0.000000,30.022214,16.532796,6.733003,45.254834
6,51.478151,30.016662,76.954532,17.625739,108.747414,30.022214,0.000000,14.177447,31.906112,39.208843
7,44.463468,16.911535,79.277992,12.110601,110.440029,16.532796,14.177447,0.000000,19.416488,52.179178
8,52.820451,18.303005,76.902536,28.542366,106.273233,6.733003,31.906112,19.416488,0.000000,62.246821
9,97.399521,69.531767,5.033223,31.144823,11.489125,45.254834,39.208843,52.179178,62.246821,0.000000


## Step3: 透過 KNN 進行補值

In [5]:

# KNN
# 設定 k 值
value_neighbors=1
from sklearn.impute import KNNImputer
#初始化: Initialize KNNImputer
imputer = KNNImputer(n_neighbors=value_neighbors)
df_filled = pd.DataFrame(imputer.fit_transform(data))
#display(data[missing_index.values==True])
#display(df_filled[missing_index.values==True])
display(df_filled)

,0,1,2,3
0,1.0,23.0,180.0,100.0
1,1.0,40.0,170.0,68.0
2,1.0,5.0,100.0,20.0
3,1.0,30.0,176.0,100.0
4,1.0,1.0,70.0,10.0
5,0.0,40.0,160.0,58.0
6,0.0,16.0,170.0,50.0
7,0.0,27.0,166.0,58.0
8,0.0,43.0,155.0,58.0
9,0.0,8.0,100.0,17.0


## K=3,weights= uniform

In [7]:
data=ori_data.copy() #複製一份資料給 data
#轉型 
#透過 preprocessing 轉換

from sklearn import preprocessing
#下面兩行程式，讓 Male=1, FeMale-0.

le = preprocessing.LabelEncoder()
data['sex']=le.fit_transform(data['sex'])
print(data['sex'])
# KNN
value_neighbors=3
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=value_neighbors)
df_filled = pd.DataFrame(imputer.fit_transform(data))
#display(data[missing_index.values==True])
#display(df_filled[missing_index.values==True])
display(df_filled)

0    1
1    1
2    1
3    1
4    1
5    0
6    0
7    0
8    0
9    0
Name: sex, dtype: int64


,0,1,2,3
0,1.0,23.0,180.000000,100.000000
1,1.0,40.0,170.000000,68.000000
2,1.0,5.0,100.000000,20.000000
3,1.0,30.0,176.000000,75.333333
4,1.0,1.0,70.000000,10.000000
5,0.0,40.0,160.000000,61.333333
6,0.0,16.0,170.000000,50.000000
7,0.0,27.0,166.000000,58.000000
8,0.0,43.0,155.000000,58.000000
9,0.0,8.0,115.333333,17.000000


## K=1,weights= distance
*由於 k=1 ，看不出 weights 的效果

In [9]:
data=ori_data.copy() #複製一份資料給 data
#轉型 
#透過 preprocessing 轉換
from sklearn import preprocessing
#下面兩行程式，讓 Male=1, FeMale-0.
#le = preprocessing.LabelEncoder()
data['sex']=le.fit_transform(data['sex'])
print(data['sex'])
# KNN
value_neighbors=1
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=value_neighbors,weights='distance')
df_filled = pd.DataFrame(imputer.fit_transform(data))
#display(data[missing_index.values==True])
#display(df_filled[missing_index.values==True])
display(df_filled)

0    1
1    1
2    1
3    1
4    1
5    0
6    0
7    0
8    0
9    0
Name: sex, dtype: int64


,0,1,2,3
0,1.0,23.0,180.0,100.0
1,1.0,40.0,170.0,68.0
2,1.0,5.0,100.0,20.0
3,1.0,30.0,176.0,100.0
4,1.0,1.0,70.0,10.0
5,0.0,40.0,160.0,58.0
6,0.0,16.0,170.0,50.0
7,0.0,27.0,166.0,58.0
8,0.0,43.0,155.0,58.0
9,0.0,8.0,100.0,17.0


## K=3,weights= distance

In [10]:

data=ori_data.copy() #複製一份資料給 data

#轉型 
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
data['sex']=le.fit_transform(data['sex'])
print(data['sex'])
# KNN
value_neighbors=3
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=value_neighbors,weights='distance')
df_filled = pd.DataFrame(imputer.fit_transform(data))
#display(data[missing_index.values==True])
#display(df_filled[missing_index.values==True])
display(df_filled)

0    1
1    1
2    1
3    1
4    1
5    0
6    0
7    0
8    0
9    0
Name: sex, dtype: int64


,0,1,2,3
0,1.0,23.0,180.000000,100.000000
1,1.0,40.0,170.000000,68.000000
2,1.0,5.0,100.000000,20.000000
3,1.0,30.0,176.000000,77.883152
4,1.0,1.0,70.000000,10.000000
5,0.0,40.0,160.000000,60.919322
6,0.0,16.0,170.000000,50.000000
7,0.0,27.0,166.000000,58.000000
8,0.0,43.0,155.000000,58.000000
9,0.0,8.0,99.462122,17.000000


## sex 類別-  類別資料轉換對於 KNN 的影響

In [12]:
data=ori_data.copy() #複製一份資料給 data
#print(data['age','height','weight'])
#ValueError: could not convert string to float: 'Male', sklearn 中的 KNN 只能處理數值型態
#假設我想以女生的資料，補女生的身高體重，男生的資料補男生的身高體重?

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
data['sex']=le.fit_transform(data['sex'])
# KNN
value_neighbors=1
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=value_neighbors)
df_filled = pd.DataFrame(imputer.fit_transform(data))
#display(data[missing_index.values==True])
display(df_filled)

,0,1,2,3
0,1.0,23.0,180.0,100.0
1,1.0,40.0,170.0,68.0
2,1.0,5.0,100.0,20.0
3,1.0,30.0,176.0,100.0
4,1.0,1.0,70.0,10.0
5,0.0,40.0,160.0,58.0
6,0.0,16.0,170.0,50.0
7,0.0,27.0,166.0,58.0
8,0.0,43.0,155.0,58.0
9,0.0,8.0,100.0,17.0


In [14]:

data=ori_data.copy() #複製一份資料給 data
data['sex'].replace({'Male': 0,'Female': 1000},inplace=True)
print(data['sex'])
# KNN
value_neighbors=1
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=value_neighbors)
df_filled = pd.DataFrame(imputer.fit_transform(data))
display(df_filled)

0       0
1       0
2       0
3       0
4       0
5    1000
6    1000
7    1000
8    1000
9    1000
Name: sex, dtype: int64


,0,1,2,3
0,0.0,23.0,180.0,100.0
1,0.0,40.0,170.0,68.0
2,0.0,5.0,100.0,20.0
3,0.0,30.0,176.0,100.0
4,0.0,1.0,70.0,10.0
5,1000.0,40.0,160.0,58.0
6,1000.0,16.0,170.0,50.0
7,1000.0,27.0,166.0,58.0
8,1000.0,43.0,155.0,58.0
9,1000.0,8.0,170.0,17.0


## 如何如何判斷補值的好壞?
運用模型預測準確度的指標，比如，比如
* MSE(Mean-Square Error)越小代表補值的準確度越高。


* Step1:取無遺失值的資料集
* Step2:隨機取幾個資料點，當作遺失值
* Step3:以step2產生的遺失值進行補值，
* Step4: 計算MSE看補值的效果。



In [15]:
data=ori_data.copy() #複製一份資料給 data
from sklearn import preprocessing
#轉型 
le = preprocessing.LabelEncoder()
data['sex']=le.fit_transform(data['sex'])
print(data['sex'])
# KNN
value_neighbors=3
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=value_neighbors,add_indicator= True)


0    1
1    1
2    1
3    1
4    1
5    0
6    0
7    0
8    0
9    0
Name: sex, dtype: int64


In [16]:
#* Step1:取無遺失值的資料集
import random
completet_data=data.dropna()
#重製index
completet_data=completet_data.reset_index(drop=True)
display(completet_data)



,sex,age,height,weight
0,1,23,180.0,100.0
1,1,40,170.0,68.0
2,1,5,100.0,20.0
3,1,1,70.0,10.0
4,0,16,170.0,50.0
5,0,27,166.0,58.0
6,0,43,155.0,58.0


In [17]:
#* Step2:隨機取 2 個資料點，當作遺失值
a = random.sample(list(completet_data.index),2) 
missing_data=completet_data.copy()
missing_data['height'].iloc[a]=np.nan
missing_data['weight'].iloc[a]=np.nan
display(missing_data)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,sex,age,height,weight
0,1,23,180.0,100.0
1,1,40,NaN,NaN
2,1,5,100.0,20.0
3,1,1,70.0,10.0
4,0,16,170.0,50.0
5,0,27,166.0,58.0
6,0,43,NaN,NaN


In [18]:
#* Step3:以step2產生的遺失值進行補值
verify_impute = pd.DataFrame(imputer.fit_transform(missing_data))
verify_impute.rename(columns={0:'sex',1:'age',2:'height',3:'weight',4:'imputed_h',5:'imputed_w'},inplace=True)
display(verify_impute)

,sex,age,height,weight,imputed_h,imputed_w
0,1.0,23.0,180.0,100.000000,0.0,0.0
1,1.0,40.0,172.0,69.333333,1.0,1.0
2,1.0,5.0,100.0,20.000000,0.0,0.0
3,1.0,1.0,70.0,10.000000,0.0,0.0
4,0.0,16.0,170.0,50.000000,0.0,0.0
5,0.0,27.0,166.0,58.000000,0.0,0.0
6,0.0,43.0,172.0,69.333333,1.0,1.0


In [19]:
display(completet_data)
display(verify_impute)

,sex,age,height,weight
0,1,23,180.0,100.0
1,1,40,170.0,68.0
2,1,5,100.0,20.0
3,1,1,70.0,10.0
4,0,16,170.0,50.0
5,0,27,166.0,58.0
6,0,43,155.0,58.0


,sex,age,height,weight,imputed_h,imputed_w
0,1.0,23.0,180.0,100.000000,0.0,0.0
1,1.0,40.0,172.0,69.333333,1.0,1.0
2,1.0,5.0,100.0,20.000000,0.0,0.0
3,1.0,1.0,70.0,10.000000,0.0,0.0
4,0.0,16.0,170.0,50.000000,0.0,0.0
5,0.0,27.0,166.0,58.000000,0.0,0.0
6,0.0,43.0,172.0,69.333333,1.0,1.0


In [20]:
# 判斷 df_filled 和 data 的距離
# height
# weight
# Step4: 計算MSE看補值的效果，可以透過這一個方法，比較不同補植策略下 MSE 的大小。


from sklearn.metrics import mean_squared_error

y_true1 =completet_data['height']
y_pred1 = verify_impute['height']
MSE_h=mean_squared_error(y_true1, y_pred1)
print(MSE_h)

#這邊的 MSE1 很大的原因，
# 是因為和遺失值旁邊的資料點數不多，對於 KNN 而言資料量要夠多，補值的效果才會比較好。
# 每次 random 的效果不同，


y_true1 =completet_data['weight']
y_pred1 = verify_impute['weight']
MSE_h=mean_squared_error(y_true1, y_pred1)
print(MSE_h)

41.857142857142854
18.603174603174587
